# Analyzing borrowers’ risk of defaulting

The purpose of this report is to examine the data set of the bank's clients' and examine their credit score, so the bank may be able to evaluate whether a client who was granted a loan by the bank will be able to repay his or her loan or will they default on the loan. 

In the report hereinafter, I will test the hypotheses that a loanee's personal status or their number of children have any affect on their ability to repay the loan or not. 

To complete the analysis and draw conclusions, I will remove duplicated rows, fix problematic values in the data, fill in missing values and group the clients' data, as required.

# Part 1 - Reading the Data and Data's General Information

**Description of the data**
- `children` - the number of children in the family
- `days_employed` - work experience in days
- `dob_years` - client's age in years
- `education` - client's education
- `education_id` - education identifier
- `family_status` - marital status
- `family_status_id` - marital status identifier
- `gender` - gender of the client
- `income_type` - type of employment
- `debt` - was there any debt on loan repayment
- `total_income` - monthly income
- `purpose` - the purpose of obtaining a loan

In [1]:
import pandas as pd

credit_score = pd.read_csv('/Users/doronerlich/Desktop/Code ME/Course Project 1/credit_scoring_eng.csv')


In [2]:
credit_score.shape

(21525, 12)

We can see that we have a lot of data to process and that the table is composed of 12 columns and 21,525 rows. 

In [3]:
credit_score.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [4]:
credit_score.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,-152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education
8,2,-6929.865299,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


From an initial glance at the data, it is possible to detect the following:
1. Some of the numbers (floats) in the "days_employed" are negative. 
2. There are duplicates in the "education" - some are in lower case letter and others all in CAPS.
3. There are several duplicates in the "purpose" column - same purpose, written in different ways (e.g. "purchase of the house", "buy real estate" and "buy residential real estate", or "to have a wedding" and "having a wedding", etc.).
4. There are missing numerical values in both the "days_employed" column and the "total_income" column.  


In [5]:
credit_score['days_employed'].describe()

count     19351.000000
mean      63046.497661
std      140827.311974
min      -18388.949901
25%       -2747.423625
50%       -1203.369529
75%        -291.095954
max      401755.400475
Name: days_employed, dtype: float64

In [6]:
credit_score['total_income'].describe()

count     19351.000000
mean      26787.568355
std       16475.450632
min        3306.762000
25%       16488.504500
50%       23202.870000
75%       32549.611000
max      362496.645000
Name: total_income, dtype: float64

Looking at the data's info, we can see that although it has 21,525 entries (and 12 columns), some columns have fewer entries - the "days_employed" and the "total_income" columns are both missing values, i.e. 19,351. It would also be noted that the "days_employed" column has negative values.
As of now, the rest of the columns in the data don't seem to be missing any values since their number of rows meet the number of entries. 

In [7]:
credit_score[credit_score['days_employed'].isnull()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


Looking over the filtered table (missing values in the "days_employed" column), it seems as though there is a correlation between missing values in the "days_employed" column and missing values on the "total_income", before further analysis - they seem symmetrical. 
However, at this initial point, this is only an assumption and it will be further investigated hereinafter by counting the missing values in all the rows with the missing values. 

In [8]:
total_rows = len(credit_score)
print('Total Rows:', total_rows)

Total Rows: 21525


In [9]:
for i in credit_score:
    if credit_score[i].isnull().sum()>0:
        print(i)

days_employed
total_income


In [10]:
print(credit_score['days_employed'].isnull().value_counts())
print()
print(credit_score['total_income'].isnull().value_counts())

False    19351
True      2174
Name: days_employed, dtype: int64

False    19351
True      2174
Name: total_income, dtype: int64


We can affirm that we only have two columns with missing values. The number of rows with missing values is 2,174 out of 21,525.

In [11]:
total_rows = len(credit_score)
missing_val = credit_score.isnull().sum()
ratio = missing_val / total_rows
print(ratio)

children            0.000000
days_employed       0.100999
dob_years           0.000000
education           0.000000
education_id        0.000000
family_status       0.000000
family_status_id    0.000000
gender              0.000000
income_type         0.000000
debt                0.000000
total_income        0.100999
purpose             0.000000
dtype: float64


In [12]:
credit_score_subset = credit_score[(credit_score['days_employed'].isnull())&(credit_score['total_income'].isnull())] 
credit_score_subset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2174 entries, 12 to 21510
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          2174 non-null   int64  
 1   days_employed     0 non-null      float64
 2   dob_years         2174 non-null   int64  
 3   education         2174 non-null   object 
 4   education_id      2174 non-null   int64  
 5   family_status     2174 non-null   object 
 6   family_status_id  2174 non-null   int64  
 7   gender            2174 non-null   object 
 8   income_type       2174 non-null   object 
 9   debt              2174 non-null   int64  
 10  total_income      0 non-null      float64
 11  purpose           2174 non-null   object 
dtypes: float64(2), int64(5), object(5)
memory usage: 220.8+ KB


As we can see from our data exploration thus far, the filtered table above has 2,174 rows and 12 columns and the number of missing values in each of the identified columns with missing values is 2,174. For now, we can conclude that the data has no other missing values (that are not numerical) and that the missing values in the two columns identified are all quantative.

It will be noted that the percentage of the missing values is roughly 10% (0.100999). 10% percent is considerably a large piece of the data and there is a correlation between two specific columns ("days_employed" and "total_income"). If a client has no days of employment and no income, one might assume that there might be some issue with the clients' employment. 

Hereinafter I will check other characteristics of the clients with missing values in the columns noted above, if there are any.
I have concluded, at least initially in this stage of the analysis, it seems that the missing values are intentional due to the correlation between two columns in the data, "days_employed" and "total_income". One might assume that clients who have no days of employment and no income are infact unemployed.  

To determine why these clients have missing values, or if it is intentional or random - I will further investigate the missing values' distribution in the group. 

In [13]:
print(credit_score[credit_score['days_employed'].isnull()]['family_status'].unique())
print()
print(credit_score[credit_score['days_employed'].isnull()]['gender'].unique())
print()
print(credit_score[credit_score['days_employed'].isnull()]['income_type'].unique())
print()
print(credit_score[credit_score['days_employed'].isnull()]['children'].unique())

['civil partnership' 'married' 'unmarried' 'widow / widower' 'divorced']

['M' 'F']

['retiree' 'civil servant' 'business' 'employee' 'entrepreneur']

[ 0  1  2 -1  3  4 20  5]


In [14]:
print(credit_score[credit_score['total_income'].isnull()]['family_status'].unique())
print()
print(credit_score[credit_score['total_income'].isnull()]['gender'].unique())
print()
print(credit_score[credit_score['total_income'].isnull()]['income_type'].unique())

['civil partnership' 'married' 'unmarried' 'widow / widower' 'divorced']

['M' 'F']

['retiree' 'civil servant' 'business' 'employee' 'entrepreneur']


From the exploration above, we can see not only that missing values in the columns "days_employed" and "total_income" are missing in similarly identified characteristics, but that there are also abnormal number of children (-1 and 20). 
It should also be noted that there is no indication that clients with missing values in the "days_employed" column and the "total_income" are unemployed.

In [15]:
credit_score[credit_score['days_employed'].isnull()]['family_status'].value_counts()

married              1237
civil partnership     442
unmarried             288
divorced              112
widow / widower        95
Name: family_status, dtype: int64

In [16]:
credit_score['family_status'].value_counts()

married              12380
civil partnership     4177
unmarried             2813
divorced              1195
widow / widower        960
Name: family_status, dtype: int64

From the exploration above we can see that the distribution of the values in the "family_status" column are similarly distributed, whether we check the distribution of the missing values in the "days_employed" column or the whole data set. 

In [17]:
credit_score[credit_score['days_employed'].isnull()]['children'].value_counts()

 0     1439
 1      475
 2      204
 3       36
 20       9
 4        7
-1        3
 5        1
Name: children, dtype: int64

In [18]:
credit_score['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

From the exploration above we can see that the distribution of the values in the "children" column are similarly distributed, whether we check the distribution of the missing values in the "days_employed" column or the whole data set. 

In [19]:
credit_score[credit_score['days_employed'].isnull()]['income_type'].value_counts()

employee         1105
business          508
retiree           413
civil servant     147
entrepreneur        1
Name: income_type, dtype: int64

In [20]:
print(credit_score['income_type'].value_counts())

employee                       11119
business                        5085
retiree                         3856
civil servant                   1459
unemployed                         2
entrepreneur                       2
student                            1
paternity / maternity leave        1
Name: income_type, dtype: int64


From the exploration above we can see that the distribution of the values in the "income_type" column are not similarly distributed, when we check the distribution of the missing values in the "days_employed" column. As we can see, there are no missing values in the "days_employed" column for clients who are unemployed, a student or on paternity/maternity leave. 

Thus it seems, initially at least, that clients' with missing values do not have specific characteristics and are not missing any other values in the data set. 

As was previously noted, the days of employment values is missing for some clients across the board. 
In addition, it seems that clients that have missing values in the "days_employed" and "total_income" columns are more likely to be employed, with no children, and either married or in a civil partnership - like the rest of the clients in the data set. In fact, in the whole data set we generally have more clients who have no children, more likely to be female that is in some stated relationship with secondary education. 

It should also be noted, that there doesn't seem to be a correlation between the clients' with the missing values in the "days_employed" and "total_income" columns and their employment status - they are all either employed, a business, a retiree or an entrepeneur. 

As the data is still being anaylized, we will not make any conclusions yet. However, since the distribution is similar whether we look at the whole data set or only the columns with missing values, it is more likely that there are no discernible patterns or specific characteristics - **that the values are missing randomly**.  

In [21]:
credit_score[credit_score['total_income'].isnull()]['education'].value_counts()

secondary education    1408
bachelor's degree       496
SECONDARY EDUCATION      67
Secondary Education      65
some college             55
Bachelor's Degree        25
BACHELOR'S DEGREE        23
primary education        19
Some College              7
SOME COLLEGE              7
Primary Education         1
PRIMARY EDUCATION         1
Name: education, dtype: int64

In [22]:
credit_score['education'].value_counts()

secondary education    13750
bachelor's degree       4718
SECONDARY EDUCATION      772
Secondary Education      711
some college             668
BACHELOR'S DEGREE        274
Bachelor's Degree        268
primary education        250
Some College              47
SOME COLLEGE              29
PRIMARY EDUCATION         17
Primary Education         15
graduate degree            4
Graduate Degree            1
GRADUATE DEGREE            1
Name: education, dtype: int64

In addition to the exploration done thus far, we can also note that there doesn't seem to be a pattern or a specific characteristic when it comes to education. The distribution is the same when we check the missing values and when we check the whole data set. 

In [23]:
credit_score[credit_score['days_employed'].isnull()]['dob_years'].value_counts()

34    69
40    66
42    65
31    65
35    64
36    63
47    59
41    59
30    58
28    57
58    56
57    56
54    55
56    54
38    54
52    53
37    53
33    51
39    51
50    51
43    50
45    50
49    50
51    50
29    50
46    48
55    48
48    46
44    44
53    44
60    39
62    38
61    38
32    37
64    37
23    36
27    36
26    35
59    34
63    29
25    23
24    21
65    20
66    20
21    18
22    17
67    16
0     10
68     9
71     5
69     5
20     5
70     3
72     2
19     1
73     1
Name: dob_years, dtype: int64

In [24]:
credit_score['dob_years'].value_counts()

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
22    183
66    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

There are also no discernible patterns when checking the distribution of the clients' age. Though it should be noted that some clients are 0 years old, this should also be considered a missing value. 

In [25]:
credit_score[credit_score['days_employed'].isnull()]['gender'].value_counts()

F    1484
M     690
Name: gender, dtype: int64

In [26]:
credit_score['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In conclusion, I did not detect any specific characteristics or patterns as to why there are symmetrical missing values in the "days_employed" and "total_income" columns. Overall, the clients with the missing values seem to be of more or less the same type as any other client in data set: either male or female (more likely female), employed, in a relationship but with no children, with secondary education between the ages of 26-60. 

In the following parts of the report I will take the following steps:
1. As noted above, all the missing values - number of days of employment, sum of total income and age - are quantitive. As such, and especially since no patterns were identified as to why they are missing, I will them with either the column's mean or median, depending on whether there are outliers identified. If there are no outliers, I will use a mean to fill in the missing value. If there are outliers, I will use the column's median.

2. Every column in the data set will be examined on whether there are duplicated, incorrect or problematic values. From our intial exploration, the following was noticeable: 
- Some clients have an incorrect number of children, such as -1 or 20. I will consider whether it was intentional or a typo and correct the values accordingly. 
- One client has no assigned gender. Since it is only one row out of more than 21,000 - I will consider removing the row entirely. 
- The "education" column has duplicate values, some in CAPS and some in lower case letter, and some are mixed. I will alter all the values to lower case letter so there will be no duplicate values in the column. 
- As noted, there are some negative values in the "days_employed" column. I will check the precentage of those values and determine whether to turn them into positive values or remove them entirely. 
- I will check if there are any duplicate rows and, depending on the number of duplicate rows, might drop them. 

# Part 2 - Fixing Incorrect Values and Removing Duplicates

**Clients' Education Column**

I will check below how many values there are in the "education" column. 

In [27]:
credit_score['education'].value_counts()

secondary education    13750
bachelor's degree       4718
SECONDARY EDUCATION      772
Secondary Education      711
some college             668
BACHELOR'S DEGREE        274
Bachelor's Degree        268
primary education        250
Some College              47
SOME COLLEGE              29
PRIMARY EDUCATION         17
Primary Education         15
graduate degree            4
Graduate Degree            1
GRADUATE DEGREE            1
Name: education, dtype: int64

We can see that there are a lot of values, 15 to be exact, but that most of them are duplicates due to the letters used (lower case and CAPS). I will turn all the values in the column to lower case letters and reprint the values. 

In [28]:
credit_score['education'] = credit_score['education'].str.lower()

In [29]:
credit_score['education'].value_counts()

secondary education    15233
bachelor's degree       5260
some college             744
primary education        282
graduate degree            6
Name: education, dtype: int64

**Clients' Number Children Column**

In [30]:
credit_score['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Once the value counts of the number of children in the data set was printed, we can see that we have incorrect values such as minus children or an excessive number of children. 

In [31]:
a = credit_score[credit_score['children'] == 20]['children'].count()
b = credit_score[credit_score['children'] == -1]['children'].count()
c = a + b
print('The total number of problematic data is', c)
print('The total number of variables in the Children column is', credit_score['children'].count())
ratio_children = c/credit_score['children'].count()
print('The precentage of problematic data is {:.0%}'.format(ratio_children))

The total number of problematic data is 123
The total number of variables in the Children column is 21525
The precentage of problematic data is 1%


The precentage of problematic data in the Children column is approximately 1% of the data, which is quite low. From the low precentage of problematic data, we can infer that it was not intentional and random, perhaps due to human error that occured while the data was typed. Which means we can either drop the problematic data or replace it. 

In [32]:
credit_score['children'] = credit_score['children'].replace({ 20 : 2 ,-1 : 1 })

In [33]:
credit_score['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

I assumed that it was probable that 20 children and -1 children were typos due to human error and replaced them with 2 and 1, respectively. 

**Days of Employment Column**

First of all, we know that there are 2,174 missing values and some negative value of days of employment. Hereinafter I will check how many of the values are in fact negative and decide how to address them, depending on the outcome.   

In [34]:
count = 0
days_employed = credit_score['days_employed']
for days_employed in credit_score['days_employed']:
    count += 1
    if days_employed < 0:
        print(days_employed)

-8437.673027760233
-4024.803753850451
-5623.422610230956
-4124.747206540018
-926.1858308789184
-2879.202052139952
-152.7795691752705
-6929.865298973741
-2188.7564450779378
-4171.483646903305
-792.7018870609315
-1846.6419410560736
-1844.9561821875543
-972.3644189900732
-1719.9342261530542
-2369.999720209008
-10038.818548915877
-1311.6041664289485
-253.68516578402495
-1766.6441376815046
-272.9813850334044
-529.1916346594236
-717.2743239418727
-1682.08343808475
-4649.910831728614
-1548.63754364046
-4488.067031277062
-176.21668833712155
-6448.810859676204
-597.8818273051091
-650.5877964026586
-1030.3626056014652
-1257.4961901986044
-4375.681384360526
-1362.0417275002387
-1039.4517405766287
-2262.712303593806
-2689.1372742032745
-3341.0678863600638
-1181.443228272809
-6953.631654563154
-3554.6715257531628
-4219.88338560805
-3480.072792773725
-2424.809748937992
-335.31863038428764
-4341.7867754100425
-2534.462390327333
-2986.20212028634
-7845.6492334208915
-1182.5477437277514
-118.8914114245

-9087.430937836874
-128.55363573881988
-1659.931156927678
-1085.319856821044
-190.44765153293227
-1951.839133979567
-4209.195789171808
-2603.5009451401465
-5223.231163664883
-609.6880979030128
-2651.061400869458
-405.0428581577137
-2081.7825004076058
-3265.3570188667004
-530.4873499428777
-1822.315338901624
-369.7002232523597
-730.8614239687781
-1417.2971020703812
-693.0424126075177
-1259.2164400131107
-1193.76442680975
-2200.833386203518
-3976.112832248061
-2916.882466654384
-3093.7004891024385
-347.5267135869046
-1372.0688054821726
-6574.970869918134
-1139.871540898328
-802.5554923527823
-2977.407625177268
-5056.171433255235
-3137.267885894231
-895.905524674093
-2392.4835002975465
-499.89523177131576
-421.50372452113226
-1608.5911266063308
-3674.1967862080487
-1190.5667425091135
-1762.1922462806897
-1796.6536521374337
-350.97659937237944
-1471.5532710536083
-505.427182098733
-914.2864898150128
-1354.4912581036328
-125.2490182668366
-1645.8672882232004
-1616.2194230043344
-1168.253700

-4788.705354159879
-1691.3624488888863
-1464.2167400153628
-5174.285935622836
-812.8831938351955
-384.74539373994935
-469.4891372227227
-1986.3264354644905
-551.0625610066053
-1522.9619659718494
-1369.869468507226
-223.2757393067604
-2701.4980472411785
-5037.596808292566
-326.9749116514291
-7065.052924316393
-2764.413357681505
-6930.325494360051
-1961.5298351564984
-4583.746059337305
-358.94024746725944
-899.7297483019606
-1157.4800197582902
-145.96702361567355
-301.10828100628135
-676.5495555680453
-387.51966516757545
-696.8697669930511
-1507.963145325387
-110.17379862902597
-8244.30459055375
-4552.411201187998
-393.69271421568016
-4235.758064449553
-4797.549936496132
-2223.70292128767
-230.9152696089617
-1643.558675710974
-4982.515805382706
-7950.456255397251
-426.8569362728449
-1224.7608827017868
-1030.99454847189
-1207.9056002831378
-386.44841879038734
-852.7028737543562
-3338.304511299257
-1342.5827353004129
-670.9371109138602
-251.6707236598449
-1294.201576914744
-1057.4749558699

-1209.2303734296868
-3934.770213142687
-5815.405650233454
-142.6363254404057
-3170.888204009769
-1765.393296887101
-1047.6929086797334
-2583.991628262609
-2428.965465844038
-2874.641081920948
-4003.5095915688985
-2907.910615964881
-445.4905057042501
-275.63658952893667
-3271.6598172718573
-4386.038610365559
-4288.714409674496
-2722.8507708057045
-2050.159363197502
-4711.267331570009
-626.5560555274502
-2697.916972813269
-899.7468321056523
-6521.03231937172
-3396.1680854528527
-2259.530440481879
-1887.4691751191456
-492.6131972738883
-1771.936782732413
-1115.6531749988137
-10836.555588720425
-628.0347855847092
-1236.5311588265906
-1503.7260287602933
-1608.7961476519524
-2198.059454180772
-255.95075573791345
-2710.8314197325767
-3582.1060208543395
-5441.0214192433
-805.0444379213608
-281.3826223487629
-853.7293072177252
-3890.0367791335298
-2083.882773599108
-1610.8875361313876
-10433.32086747994
-2853.949410583572
-703.731584398808
-553.7038466261113
-9646.854265458189
-5925.18771686205

-1460.2507095735232
-981.3787690373794
-1630.9391522246506
-661.4749610365748
-3484.9454505933973
-7653.142780889087
-2514.3219208648857
-2088.951148636376
-1427.6154931852036
-2225.4460626425107
-2042.2869963197384
-724.494559872493
-423.9380405926243
-4891.151643349373
-166.0532125743263
-10585.567475865855
-6091.244443552354
-2183.473091107773
-9725.456696975143
-837.636187548669
-319.48027074015323
-170.20158384847954
-1374.398893409153
-5669.306292900135
-3010.454664967421
-2108.359826962866
-2679.266911199166
-902.3032485188036
-1200.9103795052524
-3401.7508170926008
-1173.168301686606
-1690.1829244868802
-4643.860127829179
-2724.889726035953
-840.5561486016047
-2539.534295095165
-1423.316911633765
-1773.236413005683
-2619.03863961576
-547.8897705871535
-9959.399532274549
-117.17675028970956
-627.9621187171059
-1325.029391293339
-847.3429922863228
-874.8390537217351
-2415.350949504732
-5279.541342628562
-1767.113958621328
-2826.176623851899
-206.44912003368648
-2390.927414948688


...

-6647.141220607569
-1562.2078797433194
-1050.2232107163811
-3593.2527687012016
-4146.751902820447
-498.6183481465612
-200.1230866391991
-4392.416745211737
-1231.7075883979153
-5038.433887923286
-1408.0062137579318
-934.2929588657744
-393.1008760876265
-212.818221633485
-3072.2380721054387
-4350.094254343208
-1524.8692995442475
-1990.589347900643
-1255.043430024157
-2020.0683104853615
-3252.367160647214
-763.6311054692053
-1193.8155591233772
-532.7123237538791
-2969.4098603677508
-3073.3743445881278
-594.6251915525618
-1136.4377559821844
-985.6599687307444
-4691.274264121328
-884.1381769493811
-1121.827181951191
-780.474829768125
-1807.1022087354845
-456.9401051466722
-1712.1233049115203
-3754.962526876233
-838.6630507977134
-12564.295636721052
-150.7482694000391
-839.058178717896
-2259.950840886309
-616.7898726763848
-206.3020387289016
-3240.640456328804
-1328.2708150631472
-439.2597815970588
-3396.32330389692
-2060.1640639006096
-2026.475097211545
-207.6766685216781
-5803.543189523169

-2001.7456111581707
-1091.7424356197143
-2567.926000484372
-632.6719791426899
-1058.3436003188483
-6797.261165929843
-1333.5897900844795
-996.2328051330284
-753.5498295067682
-551.4867029672579
-3426.385817769124
-667.7374375827618
-1598.3181727086212
-4796.581760914572
-2825.4823302396007
-3944.422330594844
-2793.811356405149
-465.45073947340967
-3325.7714753308287
-548.3650040681963
-1505.9745151532086
-447.8836745183112
-4529.53249085173
-3035.319301927452
-648.3202430811551
-2625.248416682157
-1425.115952563386
-3076.6408269270646
-1361.7780793970649
-1354.3551707490894
-1437.4010582881765
-4768.355137620703
-2199.368898041587
-2660.340542155204
-7860.6788774472925
-2354.031950685562
-7906.737456599749
-4760.712449838901
-724.8307052262562
-9519.657328957472
-669.2810773243879
-746.2903366421652
-1597.7098416236583
-4622.005284750443
-230.7789337516644
-2096.542860866319
-470.8166621819264
-1124.5791718017947
-2124.5555638318983
-2887.8490351267387
-578.8235778199547
-1322.05537579

-2220.5219170347004
-2197.460718185312
-1487.037346874586
-3452.1987195237284
-13702.636708175783
-2807.9150578105127
-382.95024029550376
-1545.2842056354157
-3368.789311895883
-7995.4537732049475
-3889.818290612717
-3233.658397542254
-514.3279361526728
-3229.2337473587804
-1277.6460478971276
-274.7303262993352
-231.53373165682288
-405.1940974277213
-411.9443465446498
-3668.067316114287
-111.6767188371378
-1489.0810685851804
-450.4288859973592
-2987.004811735333
-1518.4831688388235
-709.9022761943588
-1219.2176289718695
-491.8415835743192
-405.0706675335318
-2936.038402698847
-1345.51335743934
-2180.8578203430084
-496.041757947324
-1730.5510662496802
-1842.536392188963
-302.43608153144766
-3484.967346417063
-1627.8429371227826
-239.55339374385068
-2511.9657105694496
-913.5106751206524
-1237.3633590428924
-1124.806311721351
-141.22813890179506
-1430.7850660878871
-6848.9783511281385
-6108.553490684294
-975.7129751685218
-9778.872044745314
-499.9637287702152
-2147.1394041215303
-2631.408

-4144.945839930351
-4489.276558433488
-5438.511046883361
-661.1485787135466
-2086.472121352398
-3181.931404420224
-3952.59936406196
-303.15147261877416
-686.7871042154662
-2323.813590544644
-1313.8280262121236
-3372.301024920129
-140.57283119336563
-3642.511639541525
-793.5247580845689
-8758.694572122002
-1500.9286978395533
-4137.331614974311
-503.9695161430095
-1709.1582647143928
-1339.521871434653
-2083.934423378985
-8437.66023615963
-935.6219216399148
-295.8192311739008
-1353.831132572511
-2059.601931196808
-62.10848678222075
-1279.0847043591036
-1737.695960363641
-392.58185748564586
-4093.621851322837
-3163.1315046340974
-257.7196442439495
-3907.1030497171087
-2387.0391421975164
-2410.2055342288154
-4410.059633454913
-4354.812374919271
-3062.072995231715
-1223.1720659112837
-179.34635712930702
-2935.2679954133323
-942.822974365952
-341.3540728543212
-4074.3882067694703
-1398.4347857515618
-556.165958390676
-1265.5986035307449
-1812.084853902544
-1160.4869112796382
-677.376662509611

-1445.6066452762338
-547.5787883519594
-3485.556359666788
-392.3808514520563
-3888.475152998248
-7369.734317140673
-2775.8552909574487
-2312.917644979293
-1370.2026654300494
-5541.835794752996
-204.19557672907456
-2386.416328719929
-2307.550390170796
-915.443992812599
-5111.212551997978
-1202.6654087043194
-847.9617377769458
-715.4521592022078
-981.5937229509244
-85.28272332936633
-196.71019129858223
-1872.264947953046
-245.55958537927583
-399.0752341036055
-3779.320551948116
-4089.614627891693
-1388.3526641925266
-3358.918760109553
-3135.664530706679
-775.6508811465162
-607.0296789220217
-1683.5957846313129
-3197.840587440593
-2002.957310004221
-1676.4705450630918
-1776.5900381331298
-738.0143319347696
-990.5659940005646
-824.7625412646005
-352.048655050933
-562.9634242293442
-225.90242092726265
-1290.8189346881036
-4082.735432138585
-1852.3040803935648
-113.54781526425164
-260.3119661997438
-223.4066952254204
-1928.4973679170928
-2013.7771500868785
-1268.4877281976171
-943.7500314976

In [35]:
len(credit_score[credit_score['days_employed']<0])

15906

As we can see, out of a total number of values (21,525, minus the missing values) - 15,906 are negative.

In [36]:
total_values = 21525
negative_days = len(credit_score[credit_score['days_employed']<0])
ratio_days_employed = negative_days/total_values

print('The precentage of problematic data is {:.0%}'.format(ratio_days_employed))

The precentage of problematic data is 74%


In [37]:
credit_score['days_employed'] = credit_score['days_employed'].abs()

In [38]:
credit_score['days_employed'].head(10)

0      8437.673028
1      4024.803754
2      5623.422610
3      4124.747207
4    340266.072047
5       926.185831
6      2879.202052
7       152.779569
8      6929.865299
9      2188.756445
Name: days_employed, dtype: float64

We can see that the percentage of problematic data (i.e. the negative number of days of employment) is very high - about 74%. That indicates not only that it is accidental, but that maybe it's due to an error, or technical glitch. 

Thus, I have turned the negative values into positive values, I will address the missing values in the following chapter of this report. 

As stated above, the purpose of this report is to see whether a client will default on a loan granted to them by the bank. As such, I will consider deleting the whole **column** detailing each client's number of days of employment since it is irrelavent for the purpose of this report and we have other information that may indicate whether a client is employed or not - especially since I have previously concluded that there is no pattern between the values in the "days_employed" column (or lack thereof) and the "income_type" column.   


**Clients' Age Column**

In [39]:
credit_score['dob_years'].value_counts().sort_index()

0     101
19     14
20     51
21    111
22    183
23    254
24    264
25    357
26    408
27    493
28    503
29    545
30    540
31    560
32    510
33    581
34    603
35    617
36    555
37    537
38    598
39    573
40    609
41    607
42    597
43    513
44    547
45    497
46    475
47    480
48    538
49    508
50    514
51    448
52    484
53    459
54    479
55    443
56    487
57    460
58    461
59    444
60    377
61    355
62    352
63    269
64    265
65    194
66    183
67    167
68     99
69     85
70     65
71     58
72     33
73      8
74      6
75      1
Name: dob_years, dtype: int64

As we can see, the clients' age range from 19-75, while the bulk of the ages are between 25-62. We can see that there are 101 clients whose age is 0, which is unusual and unlikely. In order to know how to fill the problematic values I will check the median of the clients' age.  

In [40]:
median_years = credit_score['dob_years'].median()
print(median_years)

42.0


We can see that the clients' median age is 42.

In [41]:
credit_score['dob_years'] = credit_score['dob_years'].replace(0, median_years)

In [42]:
credit_score['dob_years'].value_counts().sort_index()

19     14
20     51
21    111
22    183
23    254
24    264
25    357
26    408
27    493
28    503
29    545
30    540
31    560
32    510
33    581
34    603
35    617
36    555
37    537
38    598
39    573
40    609
41    607
42    698
43    513
44    547
45    497
46    475
47    480
48    538
49    508
50    514
51    448
52    484
53    459
54    479
55    443
56    487
57    460
58    461
59    444
60    377
61    355
62    352
63    269
64    265
65    194
66    183
67    167
68     99
69     85
70     65
71     58
72     33
73      8
74      6
75      1
Name: dob_years, dtype: int64

I have replaced the clients' with age "0" with the median age "42". As we can see, there are no longer clients in the data with the age "0" and the number of clients with the age "42" increaed by 101, from 597 to 698.

In [43]:
credit_score

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions
21521,0,343937.404131,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car
21522,1,2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property
21523,3,3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car


**Personal Status Column**

In [44]:
credit_score['family_status'].value_counts()

married              12380
civil partnership     4177
unmarried             2813
divorced              1195
widow / widower        960
Name: family_status, dtype: int64

I did not find that there are any problematic values in the "family_status" column.

**Clients' Gender Column**

In [45]:
credit_score['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [46]:
credit_score[credit_score['gender'] == 'XNA']['gender']

10701    XNA
Name: gender, dtype: object

In [47]:
credit_score = credit_score.drop(credit_score.index[10701])

In [48]:
credit_score['gender'].value_counts()

F    14236
M     7288
Name: gender, dtype: int64

Since there was only one row out of 21,525 rows with no assigned gender (XNA) - the row was removed from the whole data.

**Clients' Type of Income Column**

In [49]:
credit_score['income_type'].value_counts()

employee                       11119
business                        5084
retiree                         3856
civil servant                   1459
unemployed                         2
entrepreneur                       2
student                            1
paternity / maternity leave        1
Name: income_type, dtype: int64

In [50]:
print(credit_score[credit_score['income_type'] == 'unemployed']['income_type'])
print()
print(credit_score[credit_score['income_type'] == 'entrepreneur']['income_type'])
print()
print(credit_score[credit_score['income_type'] == 'paternity / maternity leave']['income_type'])
print()
print(credit_score[credit_score['income_type'] == 'student']['income_type'])

3133     unemployed
14798    unemployed
Name: income_type, dtype: object

5936     entrepreneur
18697    entrepreneur
Name: income_type, dtype: object

20845    paternity / maternity leave
Name: income_type, dtype: object

9410    student
Name: income_type, dtype: object


I did not Identify any problematic values in the "income_type" column, although there are 6 clients in four values on the outskirts of the main body of the data. Just in case, I have located the rows. Perhaps I will decide to delete them, since they don't consist a large percentage of the data and their deletion most likely won't affect the conclusion.

**Checking Data for Duplicates**

In [51]:
credit_score.duplicated().sum()

72

We can see that we have 72 duplicated rows in the data.

In [52]:
credit_score = credit_score.drop_duplicates().reset_index(drop = True)

In [53]:
credit_score.duplicated().sum()

0

In [54]:
credit_score.shape

(21452, 12)

1. The "education" column containing the clients' education was altered to lower case letters. 
2. I concluded that the problematic number of clients' children was due to typos and corrected the problematic values.
3. I turned the negative values in the "days_employed" column to positive values. 
4. While reviewing the "dob_years" column, I noticed that there were clients with an unusual age, "0". I checked what's the median age of all the clients in the data, and replaced the age "0" with the median age "42".
5. In the "gender" column there was one client without an assigned gender, since it was only one client out of 21,525 - that row was deleted.
6. 72 duplicated rows were located in the entire data and were removed. 

The changes to the data are that now there are 21,452 rows and 12 columns. 

# Part 3 - Working with Missing Values

We can create dictionaries for "education" and "education_id" and "family_status" and "family_status_id", but it doesn't seem necessary in this report.

As noted throughout this report, there are two columns in the data set with missing (null) values - "days_employed" and "total_income".

I plan to fix the "total_income" column by filling it with either a mean or a median, depending on further investigation below. I believe filling those missing values is better than ignoring them is because that an income may affect a client's ability to return a loan granted by the bank and the values are missing randomly - there is no correlation with other factors, such as the client's type of income (whether he is employed or not or a retiree). 

In regard to the the missing values in the "days_employed" column, in the previous part I fix all the negative values in the data and turned them to positive values. However, I also noted that it doesn't seem to be any significant value to the data in the column and that it does not correlate with any other of the clients' characteristics. Moreover, I don't think that the data in the column has any implications on whether a client will default on a loan or not. Thus, after further investigation, I may conclude that the column can be removed from the data set. 

### Restoring missing values in `total_income`

In this section I will address the missing values in the "total_income" column, as noted above. 

In [55]:
def assign_age_group(age):
    if age < 0 or pd.isna(age):
         return 'NA'
    elif age < 20:
        return '0-19'
    elif age < 40:
        return '20-39'
    elif age < 60:
        return '40-59'
    elif age < 40:
        return '30-39'
    elif age < 60:
        return '40-59'
    else: return '60+'

In [56]:
print(assign_age_group(60))

60+


In [57]:
credit_score['age_group'] = credit_score['dob_years'].apply(assign_age_group)

In [58]:
credit_score['age_group'].head(10)

0    40-59
1    20-39
2    20-39
3    20-39
4    40-59
5    20-39
6    40-59
7    40-59
8    20-39
9    40-59
Name: age_group, dtype: object

Since the clients' age is varied across all ages, I have created a new column containing the clients' age group.

In [59]:
credit_score.loc[credit_score['total_income'].isnull()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding,60+
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education,40-59
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate,60+
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase,40-59
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding,40-59
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21416,2,NaN,47,secondary education,1,married,0,M,business,0,NaN,purchase of a car,40-59
21422,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony,40-59
21424,0,NaN,48,bachelor's degree,0,married,0,F,business,0,NaN,building a property,40-59
21429,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate,40-59


I located rows where there no values in the "total_income" column.

In [60]:
credit_score_no_null = credit_score.dropna()

In [61]:
credit_score_no_null.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,40-59
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,20-39
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,20-39
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,20-39
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,40-59
5,0,926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house,20-39
6,0,2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions,40-59
7,0,152.779569,50,secondary education,1,married,0,M,employee,0,21731.829,education,40-59
8,2,6929.865299,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,having a wedding,20-39
9,0,2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family,40-59


In [62]:
credit_score_no_null['total_income'].mean()

26787.266687648615

In [63]:
credit_score_no_null['total_income'].median()

23201.8735

In [64]:
credit_score_no_null.groupby('family_status')['total_income'].mean()

family_status
civil partnership    26692.840381
divorced             27189.354550
married              27041.784689
unmarried            26934.069805
widow / widower      22984.208556
Name: total_income, dtype: float64

In [65]:
credit_score_no_null.groupby('family_status')['total_income'].median()

family_status
civil partnership    23185.477
divorced             23515.096
married              23389.540
unmarried            23149.028
widow / widower      20514.190
Name: total_income, dtype: float64

In [66]:
credit_score_no_null.groupby('children')['total_income'].mean()

children
0    26421.916832
1    27368.627863
2    27478.854282
3    29322.623993
4    27289.829647
5    27268.847250
Name: total_income, dtype: float64

In [67]:
credit_score_no_null.groupby('children')['total_income'].median()

children
0    23027.3350
1    23660.5630
2    23136.1155
3    25155.4480
4    24981.6340
5    29816.2255
Name: total_income, dtype: float64

In [68]:
credit_score_no_null.groupby('income_type')['total_income'].mean()

income_type
business                       32386.741818
civil servant                  27343.729582
employee                       25820.841683
entrepreneur                   79866.103000
paternity / maternity leave     8612.661000
retiree                        21940.394503
student                        15712.260000
unemployed                     21014.360500
Name: total_income, dtype: float64

In [69]:
credit_score_no_null.groupby('income_type')['total_income'].median()

income_type
business                       27571.0825
civil servant                  24071.6695
employee                       22815.1035
entrepreneur                   79866.1030
paternity / maternity leave     8612.6610
retiree                        18962.3180
student                        15712.2600
unemployed                     21014.3605
Name: total_income, dtype: float64

In [70]:
credit_score_no_null.groupby('age_group')['total_income'].mean()

age_group
0-19     16993.942462
20-39    27326.088049
40-59    27261.785611
60+      23021.639994
Name: total_income, dtype: float64

In [71]:
credit_score_no_null.groupby('age_group')['total_income'].median()

age_group
0-19     14934.901
20-39    23875.883
40-59    23432.144
60+      19761.425
Name: total_income, dtype: float64

In [72]:
credit_score_no_null.groupby('debt')['total_income'].mean()

debt
0    26848.336178
1    26096.143537
Name: total_income, dtype: float64

In [73]:
credit_score_no_null.groupby('debt')['total_income'].median()

debt
0    23225.474
1    22928.480
Name: total_income, dtype: float64

The purpose of the analysis above is to check whether there is a difference in the clients' total income rate or median (middle number of the total income) depending on either the clients' family status, their number of children, income type, age group or debt. 


From the analysis above, we can see that the mean and the median for the clients' total income is 26,787 and 23,201, respectively. In addition, we can see that the mean and median most similar to the overall mean and median are when clients have no children - 26,421 and 23,027, respectively. 

    
I will fill in the missing values with the median, grouped by the "income_type" and the "total_income" columns = 22,815.1035.
    


In [74]:
credit_score.groupby(['income_type'])['total_income'].median()


income_type
business                       27571.0825
civil servant                  24071.6695
employee                       22815.1035
entrepreneur                   79866.1030
paternity / maternity leave     8612.6610
retiree                        18962.3180
student                        15712.2600
unemployed                     21014.3605
Name: total_income, dtype: float64

In [75]:
credit_score['total_income'].fillna(credit_score.groupby(['income_type'])['total_income'].transform('median'), inplace=True)  


    
In the function above, I instructed to fill the missing values in the "total_income" column with the median 22,815.1035.
    


In [76]:
for i in credit_score:
    if credit_score[i].isnull().sum()>0:
        print(i)

days_employed


I checked what columns there in the whole data set without values. The result of that inquiry was the "days_employed" column. 

In [77]:
credit_score[credit_score['total_income'].notna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,40-59
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,20-39
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,20-39
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,20-39
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,40-59
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21447,1,4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions,40-59
21448,0,343937.404131,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car,60+
21449,1,2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property,20-39
21450,3,3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car,20-39


In [78]:
credit_score.loc[21415:21438, 'total_income']

21415    20623.5980
21416    27571.0825
21417     5562.8740
21418    24883.3440
21419    47237.4740
21420    23600.4160
21421    28219.1350
21422    22815.1035
21423    19102.8190
21424    27571.0825
21425    38522.8120
21426    25208.5050
21427    12450.1270
21428    13797.1400
21429    22815.1035
21430    42280.1600
21431    12890.6110
21432    12070.3990
21433    23286.7190
21434    15708.8450
21435    11622.1750
21436    11684.6500
21437    22815.1035
21438    22410.9560
Name: total_income, dtype: float64

    
I have located rows in range where there were previously no values in the "total_income" column to check if the value was replaced with the median 22,815.1035. As we can see, rows 21416, 21422, 21424, 21429 and 21437, were previously NaN and are now no longer null, but with the total income of 22,815.1035.
    


In [79]:
credit_score.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21452 entries, 0 to 21451
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21452 non-null  int64  
 1   days_employed     19350 non-null  float64
 2   dob_years         21452 non-null  int64  
 3   education         21452 non-null  object 
 4   education_id      21452 non-null  int64  
 5   family_status     21452 non-null  object 
 6   family_status_id  21452 non-null  int64  
 7   gender            21452 non-null  object 
 8   income_type       21452 non-null  object 
 9   debt              21452 non-null  int64  
 10  total_income      21452 non-null  float64
 11  purpose           21452 non-null  object 
 12  age_group         21452 non-null  object 
dtypes: float64(2), int64(5), object(6)
memory usage: 2.1+ MB


Using both a table of the data where there are no values in the "total_income" column and the whole data set's info, it is clear that all missing values in the "total_income" column were replaced with the mean of the "total_income" of the clients without children. 

###  Restoring values in `days_employed`

In this section I will address the missing values in the "days_employed" column, as noted above. 

In [80]:
credit_score.loc[credit_score['days_employed'].isnull()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,18962.3180,to have a wedding,60+
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,24071.6695,education,40-59
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,18962.3180,building a real estate,60+
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,24071.6695,second-hand car purchase,40-59
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,18962.3180,to have a wedding,40-59
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21416,2,NaN,47,secondary education,1,married,0,M,business,0,27571.0825,purchase of a car,40-59
21422,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,22815.1035,wedding ceremony,40-59
21424,0,NaN,48,bachelor's degree,0,married,0,F,business,0,27571.0825,building a property,40-59
21429,1,NaN,42,secondary education,1,married,0,F,employee,0,22815.1035,building a real estate,40-59


I located rows where there no values in the "days_employed" column.

In [81]:
credit_score_no_null['days_employed'].mean()

66918.06514084802

In [82]:
credit_score_no_null['days_employed'].median()

2194.218767670908

In [83]:
credit_score_no_null.groupby('family_status')['days_employed'].mean()

family_status
civil partnership     58411.268077
divorced              68816.335483
married               63312.782890
unmarried             47072.691647
widow / widower      205636.887848
Name: days_employed, dtype: float64

In [84]:
credit_score_no_null.groupby('family_status')['days_employed'].median()

family_status
civil partnership      1943.995523
divorced               2401.954568
married                2304.964439
unmarried              1462.009287
widow / widower      337017.713307
Name: days_employed, dtype: float64

In [85]:
credit_score_no_null.groupby('children')['days_employed'].mean()

children
0    92526.171104
1    23454.811112
2     6686.606176
3     9338.376355
4    13863.043444
5     1432.348601
Name: days_employed, dtype: float64

In [86]:
credit_score_no_null.groupby('children')['days_employed'].median()

children
0    2625.852176
1    1669.174020
2    1678.132084
3    1765.066044
4    1905.879025
5    1231.571486
Name: days_employed, dtype: float64

In [87]:
credit_score_no_null.groupby('income_type')['days_employed'].mean()

income_type
business                         2111.470404
civil servant                    3399.896902
employee                         2326.499216
entrepreneur                      520.848083
paternity / maternity leave      3296.759962
retiree                        365003.491245
student                           578.751554
unemployed                     366413.652744
Name: days_employed, dtype: float64

In [88]:
credit_score_no_null.groupby('income_type')['days_employed'].median()

income_type
business                         1546.333214
civil servant                    2689.368353
employee                         1574.202821
entrepreneur                      520.848083
paternity / maternity leave      3296.759962
retiree                        365213.306266
student                           578.751554
unemployed                     366413.652744
Name: days_employed, dtype: float64

In [89]:
credit_score_no_null.groupby('age_group')['days_employed'].mean()

age_group
0-19        633.678086
20-39      3411.876229
40-59     68271.549905
60+      286544.143436
Name: days_employed, dtype: float64

In [90]:
credit_score_no_null.groupby('age_group')['days_employed'].median()

age_group
0-19        724.492610
20-39      1320.557284
40-59      2840.674560
60+      355229.618218
Name: days_employed, dtype: float64

In [91]:
credit_score_no_null.groupby('debt')['days_employed'].mean()

debt
0    68946.730621
1    43959.666946
Name: days_employed, dtype: float64

In [92]:
credit_score_no_null.groupby('debt')['days_employed'].median()

debt
0    2267.408539
1    1498.192818
Name: days_employed, dtype: float64

In the analysis above, it is noticeable that the distribution of the days of employment of clients' across several parameters that there are significant outliers in the data. For this reason, I believe it will be more appropriate to fill in the missing values with the days_employed median. 

In [93]:
credit_score['days_employed'].fillna(credit_score.groupby(['income_type'])['days_employed'].transform('median'), inplace=True)  


    
In the function above, I instructed to fill the missing values in the "days_employed" column with the median, grouped by the columns "days_employed" and "income_type".
    

In [94]:
for i in credit_score:
    if credit_score[i].isnull().sum()>0:
        print(i)

I checked what columns there in the whole data set without values. The result of that inquiry was that there are no columns without values.

In [95]:
credit_score[credit_score['days_employed'].notna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,40-59
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,20-39
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,20-39
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,20-39
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,40-59
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21447,1,4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions,40-59
21448,0,343937.404131,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car,60+
21449,1,2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property,20-39
21450,3,3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car,20-39


In [96]:
credit_score.loc[21415:21438, 'days_employed']

21415       354.755760
21416      1546.333214
21417      1867.006757
21418       173.954460
21419      1618.645283
21420      1214.858885
21421       960.095747
21422      1574.202821
21423       759.680851
21424      1546.333214
21425      1330.627998
21426      9929.015065
21427       578.082757
21428    334343.096304
21429      1574.202821
21430      3096.881131
21431    355235.728158
21432    338904.866406
21433      1556.249906
21434        79.832064
21435    386497.714078
21436    362161.054124
21437      1574.202821
21438       612.569129
Name: days_employed, dtype: float64

    
I have located rows in range where there were previously no values in the "days_employed" column to check if the value was replaced with the median. As we can see, rows 21416, 21422, 21424, 21429 and 21437, were previously NaN and are now no longer null.
    


In [97]:
credit_score.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21452 entries, 0 to 21451
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21452 non-null  int64  
 1   days_employed     21452 non-null  float64
 2   dob_years         21452 non-null  int64  
 3   education         21452 non-null  object 
 4   education_id      21452 non-null  int64  
 5   family_status     21452 non-null  object 
 6   family_status_id  21452 non-null  int64  
 7   gender            21452 non-null  object 
 8   income_type       21452 non-null  object 
 9   debt              21452 non-null  int64  
 10  total_income      21452 non-null  float64
 11  purpose           21452 non-null  object 
 12  age_group         21452 non-null  object 
dtypes: float64(2), int64(5), object(6)
memory usage: 2.1+ MB



Using both a table of the data where there are no values in the "days_employed" column and the whole data set's info, it is clear that all missing values in the "days_employed" column were replaced with the median. 
    


Once this analysis was completed, I have concluded that a client's days of employment are irrelevant to the purpose of this report (to check if a client's marital status or number of chilren might impact whether they will default on their loan). Thus, in the following function I will remove the "days_employed" column entirely from the data set. 

In [98]:
# the reviewer's code
credit_score['days_employed'].describe()

count     21452.000000
mean      67065.304480
std      139204.834203
min          24.141633
25%        1023.617476
50%        1996.223132
75%        5321.001947
max      401755.400475
Name: days_employed, dtype: float64

In [99]:
credit_score = credit_score.drop('days_employed', 1)

/var/folders/zk/dmfz7khd1d7527x4jk58bf6r0000gn/T/ipykernel_22136/1636005335.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  credit_score = credit_score.drop('days_employed', 1)


In [100]:
credit_score.head(10)

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,40-59
1,1,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,20-39
2,0,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,20-39
3,3,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,20-39
4,0,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,40-59
5,0,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house,20-39
6,0,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions,40-59
7,0,50,secondary education,1,married,0,M,employee,0,21731.829,education,40-59
8,2,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,having a wedding,20-39
9,0,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family,40-59


In [101]:
credit_score.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21452 entries, 0 to 21451
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21452 non-null  int64  
 1   dob_years         21452 non-null  int64  
 2   education         21452 non-null  object 
 3   education_id      21452 non-null  int64  
 4   family_status     21452 non-null  object 
 5   family_status_id  21452 non-null  int64  
 6   gender            21452 non-null  object 
 7   income_type       21452 non-null  object 
 8   debt              21452 non-null  int64  
 9   total_income      21452 non-null  float64
 10  purpose           21452 non-null  object 
 11  age_group         21452 non-null  object 
dtypes: float64(1), int64(5), object(6)
memory usage: 2.0+ MB


We can see that our data has 12 columns and that the "days_employed" column was deleted. 

# Part 3 - Categorization of Data

In this report, we need to check whether the clients' number of children or their marital status might have an impact on their ability to return a loan granted to them by the bank - in other words, will they default on the loan. 

As we have seen, it does not appear that we need to catogarize either the "children" column or the "family_status" column in the data set since they are concise and clear. However, there are two other columns that might have an impact on a client's ability to repay a loan: their income and why  they took the loan to begin with (i.e. the purpose). Hereinafter I will categorize these two columns. 


In [102]:
credit_score_ref = credit_score[['total_income','purpose']]
print(credit_score_ref)

       total_income                  purpose
0         40620.102    purchase of the house
1         17932.802             car purchase
2         23341.752    purchase of the house
3         42820.568  supplementary education
4         25378.572        to have a wedding
...             ...                      ...
21447     35966.698     housing transactions
21448     24959.969        purchase of a car
21449     14347.610                 property
21450     39054.888        buying my own car
21451     13127.587             to buy a car

[21452 rows x 2 columns]


**The Purpose for Taking the Loan**

In [103]:
credit_score['purpose'].unique()

array(['purchase of the house', 'car purchase', 'supplementary education',
       'to have a wedding', 'housing transactions', 'education',
       'having a wedding', 'purchase of the house for my family',
       'buy real estate', 'buy commercial real estate',
       'buy residential real estate', 'construction of own property',
       'property', 'building a property', 'buying a second-hand car',
       'buying my own car', 'transactions with commercial real estate',
       'building a real estate', 'housing',
       'transactions with my real estate', 'cars', 'to become educated',
       'second-hand car purchase', 'getting an education', 'car',
       'wedding ceremony', 'to get a supplementary education',
       'purchase of my own house', 'real estate transactions',
       'getting higher education', 'to own a car', 'purchase of a car',
       'profile education', 'university education',
       'buying property for renting out', 'to buy a car',
       'housing renovation', 'going

From scanning the array of purposes given by the banks clients, it is noticable that there are several main purposes for taking loans: 
- Wedding Expenses; 
- Residential Real Estate; 
- Commercial Real Estate; 
- Education;
- Vehicle Financing. 

In [104]:
def purpose_func(line):
  
    words = nltk.word_tokenize(line)
    lemmas = [wordnet_lemma.lemmatize(w, pos = 'n') for w in words]
    lemmas=[l.lower() for l in lemmas]
    
    if any(word in lemmas for word in Wedding_Category):
        return 'Wedding Expenses'
    elif  any(word in lemmas for word in Real_Estate_Category):
        return 'Residential/Commercial Real Estate'
    elif  any(word in lemmas for word in Education_Category):
        return 'Education'
    elif  any(word in lemmas for word in Car_Category):
        return 'Car Financing'
    else:
        
        return 'other'

In [105]:
import nltk
from nltk.stem import WordNetLemmatizer
from collections import Counter

wordnet_lemma = WordNetLemmatizer()

In [106]:
purpose='Wedding Expenses,Residential/Commercial Real Estate,Education,Car Financing'
words = nltk.word_tokenize(purpose)
print(words)
lemmas = [wordnet_lemma.lemmatize(w, pos = 'n') for w in words]
print(lemmas) 

['Wedding', 'Expenses', ',', 'Residential/Commercial', 'Real', 'Estate', ',', 'Education', ',', 'Car', 'Financing']
['Wedding', 'Expenses', ',', 'Residential/Commercial', 'Real', 'Estate', ',', 'Education', ',', 'Car', 'Financing']


In [107]:
lemmas_list_all = []

for purpose in credit_score['purpose'].unique():
    words = nltk.word_tokenize(purpose)
    lemmas = [wordnet_lemma.lemmatize(w, pos = 'n') for w in words]
    lemmas=[l.lower() for l in lemmas]
    for i in lemmas:
        lemmas_list_all.append(i)

In [108]:
lemmas_list_all

['purchase',
 'of',
 'the',
 'house',
 'car',
 'purchase',
 'supplementary',
 'education',
 'to',
 'have',
 'a',
 'wedding',
 'housing',
 'transaction',
 'education',
 'having',
 'a',
 'wedding',
 'purchase',
 'of',
 'the',
 'house',
 'for',
 'my',
 'family',
 'buy',
 'real',
 'estate',
 'buy',
 'commercial',
 'real',
 'estate',
 'buy',
 'residential',
 'real',
 'estate',
 'construction',
 'of',
 'own',
 'property',
 'property',
 'building',
 'a',
 'property',
 'buying',
 'a',
 'second-hand',
 'car',
 'buying',
 'my',
 'own',
 'car',
 'transaction',
 'with',
 'commercial',
 'real',
 'estate',
 'building',
 'a',
 'real',
 'estate',
 'housing',
 'transaction',
 'with',
 'my',
 'real',
 'estate',
 'car',
 'to',
 'become',
 'educated',
 'second-hand',
 'car',
 'purchase',
 'getting',
 'an',
 'education',
 'car',
 'wedding',
 'ceremony',
 'to',
 'get',
 'a',
 'supplementary',
 'education',
 'purchase',
 'of',
 'my',
 'own',
 'house',
 'real',
 'estate',
 'transaction',
 'getting',
 'higher'

In [109]:
Counter(lemmas_list_all)

Counter({'purchase': 6,
         'of': 5,
         'the': 2,
         'house': 3,
         'car': 9,
         'supplementary': 2,
         'education': 7,
         'to': 6,
         'have': 1,
         'a': 9,
         'wedding': 3,
         'housing': 3,
         'transaction': 4,
         'having': 1,
         'for': 2,
         'my': 4,
         'family': 1,
         'buy': 4,
         'real': 7,
         'estate': 7,
         'commercial': 2,
         'residential': 1,
         'construction': 1,
         'own': 4,
         'property': 4,
         'building': 2,
         'buying': 3,
         'second-hand': 2,
         'with': 2,
         'become': 1,
         'educated': 1,
         'getting': 2,
         'an': 1,
         'ceremony': 1,
         'get': 1,
         'higher': 1,
         'profile': 1,
         'university': 2,
         'renting': 1,
         'out': 1,
         'renovation': 1,
         'going': 1})

In the cells above, I have created a function with 4 categories: Wedding Category, Real Estate Category, Education Category and Car Category. I printed out the words in the "purpose" column and counted how many times each word appears in the column. 

In [110]:
Wedding_Category=['wedding','ceremony']
Real_Estate_Category=['house','housing','real','estate','commercial','residential','construction','property','building','renting','renovation']
Education_Category=['university','higher','educated','education','supplementary']
Car_Category=['car']

In [111]:
credit_score['Loan_Purpose_Group'] = credit_score['purpose'].apply(purpose_func)


    
In the cells above I selected a few words that are either most common or are associated with the specific category and created a new column "Loan_Purpose_Group" and applied the function.
    


In [112]:
credit_score['Loan_Purpose_Group'].count()

21452

In [113]:
credit_score.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21452 entries, 0 to 21451
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   children            21452 non-null  int64  
 1   dob_years           21452 non-null  int64  
 2   education           21452 non-null  object 
 3   education_id        21452 non-null  int64  
 4   family_status       21452 non-null  object 
 5   family_status_id    21452 non-null  int64  
 6   gender              21452 non-null  object 
 7   income_type         21452 non-null  object 
 8   debt                21452 non-null  int64  
 9   total_income        21452 non-null  float64
 10  purpose             21452 non-null  object 
 11  age_group           21452 non-null  object 
 12  Loan_Purpose_Group  21452 non-null  object 
dtypes: float64(1), int64(5), object(7)
memory usage: 2.1+ MB


In [114]:
credit_score.head(10)

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,Loan_Purpose_Group
0,1,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,40-59,Residential/Commercial Real Estate
1,1,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,20-39,Car Financing
2,0,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,20-39,Residential/Commercial Real Estate
3,3,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,20-39,Education
4,0,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,40-59,Wedding Expenses
5,0,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house,20-39,Residential/Commercial Real Estate
6,0,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions,40-59,Residential/Commercial Real Estate
7,0,50,secondary education,1,married,0,M,employee,0,21731.829,education,40-59,Education
8,2,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,having a wedding,20-39,Wedding Expenses
9,0,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family,40-59,Residential/Commercial Real Estate



    
In the cells above, I have checked how many values there are in the "Loan_Purpose_Group" column and the total number of values to check there are no missing values. Then, I printed the first 10 rows of the whole data set. 
    


**Clients' Total Income**

In the cells below I will categorize the numerical values in the "total_income" column. 

In [115]:
credit_score['total_income']

0        40620.102
1        17932.802
2        23341.752
3        42820.568
4        25378.572
           ...    
21447    35966.698
21448    24959.969
21449    14347.610
21450    39054.888
21451    13127.587
Name: total_income, Length: 21452, dtype: float64

In [116]:
credit_score['total_income'].value_counts()

22815.1035    1069
27571.0825     501
18962.3180     387
24071.6695     145
79866.1030       2
              ... 
27020.8950       1
23686.8350       1
9606.2940        1
28156.7620       1
13127.5870       1
Name: total_income, Length: 19350, dtype: int64

In [117]:
credit_score['total_income'].describe()

count     21452.000000
mean      26450.950089
std       15710.614040
min        3306.762000
25%       17217.441750
50%       22815.103500
75%       31331.122000
max      362496.645000
Name: total_income, dtype: float64

In [118]:
credit_score['total_income'].median()

22815.103499999997

As we can see we have a varied total of income, a total of 19,348 different values that range from 3,306 to 362,496. 
I have decided to group the clients' income into 4 categories:
* Low Income - will range from the minimum income to an income of 5,000.
* Below Average Income - will range from the low income to an income of 20,000.
* Average Income - Approximately the average income of 26,751 - from 20,000 to 35,000.
* Above Average Income - will range from the average income to 99,000.
* High Income - will range from 100,000 upwards. 

In [119]:
def assign_income_group(income):
    if income < 0 or pd.isna(income):
         return 'NA'
    elif income < 5000:
        return 'Low Income'
    elif income < 20000:
        return 'Below Average Income'
    elif income < 35000:
        return 'Average Income'
    elif income < 99000:
        return 'Above Average Income'
    else: return 'High Income'

In [120]:
print(assign_income_group(27000))

Average Income


I have checked that the function works.

In [121]:
credit_score['Income_Group'] = credit_score['total_income'].apply(assign_income_group)


    
I have created a new column with the assigned income groups: Income_Group. Below I will check the value count in the column and whether there are any missing values and a sample of the data set.
    


In [122]:
credit_score['Income_Group'].count()

21452

In [123]:
credit_score.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21452 entries, 0 to 21451
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   children            21452 non-null  int64  
 1   dob_years           21452 non-null  int64  
 2   education           21452 non-null  object 
 3   education_id        21452 non-null  int64  
 4   family_status       21452 non-null  object 
 5   family_status_id    21452 non-null  int64  
 6   gender              21452 non-null  object 
 7   income_type         21452 non-null  object 
 8   debt                21452 non-null  int64  
 9   total_income        21452 non-null  float64
 10  purpose             21452 non-null  object 
 11  age_group           21452 non-null  object 
 12  Loan_Purpose_Group  21452 non-null  object 
 13  Income_Group        21452 non-null  object 
dtypes: float64(1), int64(5), object(8)
memory usage: 2.3+ MB


In [124]:
credit_score.head(10)

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,Loan_Purpose_Group,Income_Group
0,1,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,40-59,Residential/Commercial Real Estate,Above Average Income
1,1,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,20-39,Car Financing,Below Average Income
2,0,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,20-39,Residential/Commercial Real Estate,Average Income
3,3,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,20-39,Education,Above Average Income
4,0,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,40-59,Wedding Expenses,Average Income
5,0,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house,20-39,Residential/Commercial Real Estate,Above Average Income
6,0,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions,40-59,Residential/Commercial Real Estate,Above Average Income
7,0,50,secondary education,1,married,0,M,employee,0,21731.829,education,40-59,Education,Average Income
8,2,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,having a wedding,20-39,Wedding Expenses,Below Average Income
9,0,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family,40-59,Residential/Commercial Real Estate,Average Income


In [125]:
credit_score.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21452 entries, 0 to 21451
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   children            21452 non-null  int64  
 1   dob_years           21452 non-null  int64  
 2   education           21452 non-null  object 
 3   education_id        21452 non-null  int64  
 4   family_status       21452 non-null  object 
 5   family_status_id    21452 non-null  int64  
 6   gender              21452 non-null  object 
 7   income_type         21452 non-null  object 
 8   debt                21452 non-null  int64  
 9   total_income        21452 non-null  float64
 10  purpose             21452 non-null  object 
 11  age_group           21452 non-null  object 
 12  Loan_Purpose_Group  21452 non-null  object 
 13  Income_Group        21452 non-null  object 
dtypes: float64(1), int64(5), object(8)
memory usage: 2.3+ MB


**Distribution of the New Categories in the Data Set**

In [126]:
credit_score['Loan_Purpose_Group'].value_counts()

Residential/Commercial Real Estate    10810
Car Financing                          4306
Education                              4013
Wedding Expenses                       2323
Name: Loan_Purpose_Group, dtype: int64

In [127]:
credit_score['Income_Group'].value_counts()

Average Income          9628
Below Average Income    7729
Above Average Income    3966
High Income              103
Low Income                26
Name: Income_Group, dtype: int64

In the cells above, we can see the distribution of clients among our new categories. For example, the most common reason for the taking a loan is real estate, either residential or commercial, and the least common reason is for various wedding explanses. We can also note that most of the clients have an income that ranges between 5,000-35,000.


It should be noted, that the group of clients with either high or low income consists of only 103 and 26 clients, respectively. 
    


# Part 4 - Renaming Data Set Columns

Now that we have all the required columns I will rename the Data Set's columns.

In [128]:
credit_score.rename(columns={'children':'No._of_Children', 'days_employed':'Days_of_Employment', 'dob_years':'Age', 'education':'Education', 'education_id':'Education_ID', 'family_status':'Personal_Status', 'family_status_id':'Personal_Status_ID', 'gender':'Gender', 'income_type':'Employment_Status', 'debt':'Reported_Debt_ID', 'total_income':'Total_Income', 'purpose':'Loan_Purpose', 'age_group':'Age_Group'}, inplace=True)

In [129]:
credit_score.head()

,No._of_Children,Age,Education,Education_ID,Personal_Status,Personal_Status_ID,Gender,Employment_Status,Reported_Debt_ID,Total_Income,Loan_Purpose,Age_Group,Loan_Purpose_Group,Income_Group
0,1,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,40-59,Residential/Commercial Real Estate,Above Average Income
1,1,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,20-39,Car Financing,Below Average Income
2,0,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,20-39,Residential/Commercial Real Estate,Average Income
3,3,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,20-39,Education,Above Average Income
4,0,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,40-59,Wedding Expenses,Average Income



    
Hereinafter I will check that the columns' names were altered.
    

In [130]:
credit_score.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21452 entries, 0 to 21451
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   No._of_Children     21452 non-null  int64  
 1   Age                 21452 non-null  int64  
 2   Education           21452 non-null  object 
 3   Education_ID        21452 non-null  int64  
 4   Personal_Status     21452 non-null  object 
 5   Personal_Status_ID  21452 non-null  int64  
 6   Gender              21452 non-null  object 
 7   Employment_Status   21452 non-null  object 
 8   Reported_Debt_ID    21452 non-null  int64  
 9   Total_Income        21452 non-null  float64
 10  Loan_Purpose        21452 non-null  object 
 11  Age_Group           21452 non-null  object 
 12  Loan_Purpose_Group  21452 non-null  object 
 13  Income_Group        21452 non-null  object 
dtypes: float64(1), int64(5), object(8)
memory usage: 2.3+ MB


# Part 5 - Checking the Hypotheses


In [131]:
debt_count = credit_score['Reported_Debt_ID'].count()
credit_score['Reported_Debt_ID'].count()

21452

In [132]:
credit_score['Reported_Debt_ID'].value_counts()

0    19711
1     1741
Name: Reported_Debt_ID, dtype: int64

We can see that most of the clients' in the Data set did not report bring in debt by being behind on loan payments.

In [133]:
debt_rate = 1741/debt_count
print("Total Clients' Debt Rate is {:.0%}".format(debt_rate))

Total Clients' Debt Rate is 8%


**Correlation Between Clients' Age Group and Paying Loan Back on Time** 

In [134]:
credit_score.groupby(['Age_Group'])['Reported_Debt_ID'].value_counts()

Age_Group  Reported_Debt_ID
0-19       0                     13
           1                      1
20-39      0                   7927
           1                    900
40-59      0                   9394
           1                    717
60+        0                   2377
           1                    123
Name: Reported_Debt_ID, dtype: int64

In [135]:
credit_score.groupby(['Age_Group'])['Reported_Debt_ID'].agg(['count','mean']).reset_index().sort_values(by='mean')

,Age_Group,count,mean
3,60+,2500,0.049200
2,40-59,10111,0.070913
0,0-19,14,0.071429
1,20-39,8827,0.101960


From the analysis above, it is noticeable that most of the clients are between the ages 40-59. However, the age group with the highest default rate is the second highest group, aged 20-39, at 10%. 

**Correlation Between Clients' No. of Children and Paying Loan Back on Time** 

In [136]:
credit_score.groupby(['No._of_Children'])['Reported_Debt_ID'].value_counts()

No._of_Children  Reported_Debt_ID
0                0                   13026
                 1                    1063
1                0                    4410
                 1                     445
2                0                    1926
                 1                     202
3                0                     303
                 1                      27
4                0                      37
                 1                       4
5                0                       9
Name: Reported_Debt_ID, dtype: int64

In [137]:
credit_score.groupby(['No._of_Children'])['Reported_Debt_ID'].agg(['count','mean']).reset_index().sort_values(by='mean')

,No._of_Children,count,mean
5,5,9,0.000000
0,0,14089,0.075449
3,3,330,0.081818
1,1,4855,0.091658
2,2,2128,0.094925
4,4,41,0.097561


From the analysis above, it is noticeable that most of the clients have no children, and their debt rate is 7% - the second lowest rate. However, clients with the most number of children have never reported being in debt while clients with 1-4 children have a debt rate of 8%-9%. 


On a side note, it should be noted that we cannot draw any conclusions concerning clients we more than 4 children, since in the data set we only have 9 clients with more than 4 children. There is not enought data on clients with more than 4 children. 
    


In light of the analysis above, there is no apparnt correlation between number of children and debt rate. 

**Correlation Between Clients' Personal Status and Paying Loan Back on Time** 

In [138]:
credit_score.groupby(['Personal_Status'])['Reported_Debt_ID'].value_counts()

Personal_Status    Reported_Debt_ID
civil partnership  0                    3761
                   1                     388
divorced           0                    1110
                   1                      85
married            0                   11408
                   1                     931
unmarried          0                    2536
                   1                     274
widow / widower    0                     896
                   1                      63
Name: Reported_Debt_ID, dtype: int64

In [139]:
credit_score.groupby(['Personal_Status'])['Reported_Debt_ID'].agg(['count','mean']).reset_index().sort_values(by='mean')

,Personal_Status,count,mean
4,widow / widower,959,0.065693
1,divorced,1195,0.071130
2,married,12339,0.075452
0,civil partnership,4149,0.093517
3,unmarried,2810,0.097509


From the analysis above, we can note that most of the clients in the data set are either married or in another kind of partnership. Other clients are either widowed, divorced or unmarried. Clients who are widowed reported the lowest debt rate, at 6%. However, divorced and married clients have approximately similar debt rate at 7% - as do clients who are either in a civil partnership or unmarried (9% each). 

Thus, it seems that clients' personal status has no bearing on their ability to repay loans on time. 

**Correlation Between Clients' Income Group and Paying Loan Back on Time** 

In [140]:
credit_score.groupby(['Income_Group'])['Reported_Debt_ID'].value_counts()

Income_Group          Reported_Debt_ID
Above Average Income  0                   3683
                      1                    283
Average Income        0                   8820
                      1                    808
Below Average Income  0                   7088
                      1                    641
High Income           0                     96
                      1                      7
Low Income            0                     24
                      1                      2
Name: Reported_Debt_ID, dtype: int64

In [141]:
credit_score.groupby(['Income_Group'])['Reported_Debt_ID'].agg(['count','mean']).reset_index().sort_values(by='mean')

,Income_Group,count,mean
3,High Income,103,0.067961
0,Above Average Income,3966,0.071357
4,Low Income,26,0.076923
2,Below Average Income,7729,0.082934
1,Average Income,9628,0.083922


From the analysis above, it is noted that clients with average income, between 20,000 and 35,000, consists the highest share of the data set and also have the highest debt rate at 8%. The second largest income group, "Below Average Income" (with reported income between 5,000-20,000), also have the second highest debt rate - also at 8%. Clients with low income (less than 5,000) is the smallest group in the data set and have the third highest debt rate. we can note that most of the clients in the data set are either married or in another kind of partnership.  

Thus, it seems that clients' total income has affect on their ability to repay loans on time - the higher the income, the lower debt rate. 

**How Clients' Reason for Taking a Loan Affects Default Rate** 

In [142]:
credit_score.groupby(['Loan_Purpose_Group'])['Reported_Debt_ID'].value_counts()

Loan_Purpose_Group                  Reported_Debt_ID
Car Financing                       0                    3903
                                    1                     403
Education                           0                    3643
                                    1                     370
Residential/Commercial Real Estate  0                   10028
                                    1                     782
Wedding Expenses                    0                    2137
                                    1                     186
Name: Reported_Debt_ID, dtype: int64

In [143]:
credit_score.groupby(['Loan_Purpose_Group'])['Reported_Debt_ID'].agg(['count','mean']).reset_index().sort_values(by='mean')

,Loan_Purpose_Group,count,mean
2,Residential/Commercial Real Estate,10810,0.072340
3,Wedding Expenses,2323,0.080069
1,Education,4013,0.092200
0,Car Financing,4306,0.093590


We can see that most of the clients in the data set's reason for taking the loan in the first place was connected to either residential or commercial real estate. However, this group also records the lowest debt rate at 7%. The debt rate is higher for clients whose reason for taking a loan was connected either to wedding expenses, education or car financing (either purchase or repair), who also consist about half of the clients together - at 8% and 9% respectively. 

Thus, it is concluded that there is little correlation between a client's reason for taking a loan and his or her ability to repay the loan on time. 

# Part 5 - General Conclusion 

In this report, we started with a data set consisting of 21,525 rows and 12 columns, with both missing values and duplicated data in the data set. 

In regards to missing values - there were clients with 0 age, no days of employment and no value of total income:
* I replaced those clients' age with the median age in the data set - 42. 
* I found that the missing values in the days of employment and the total income column were symmetrical and random (distributed across all other clients' characteristics).  
* In the days of employment column there were also negative values. Before filling in the missing values, I assumed it was a technical error and transformed all the negative values into positive values and since I saw that there were outliers in the data, filled the missing values with the median, grouped by the "days_employed" and "income_type" columns.
* The last missing values were in the clients' total income column. i checked the column's mean and median, and filled the missing values with the median, grouped by the "income_type" and "total_income" columns.
    


Once I fixed the values in the days of employment column and filled in the missing values, I decided that the data in the column has no impact on the out come of the report or the clients' other characteristics - I deleted the column. 

Other problematic values detected in the data set:
* There were duplicate values in the education column, since some statuses were in CAPS, others in lower case letter and some a mix of both (a total of 15 value counts). I altered the values in the column so the various statuses were in lower case letters and there were no duplicate values in the column - to 5 value counts. 
* There were clients with either a negative number of children or an excessive number of children, -1 and 20. I assumed it was a typo and altered the values to 1 and 2, respectively. 
* One client out of the whole data set did not have an assigned gender (either male of female) - since it was only one client, I removed the row. 
* I checked for duplicate rows and located 72 duplicated rows and since I deemed their number as insignificant - I removed them. 
 
    
I ended the preprocessing of the data set with 21,452 rows and no "days_employed" column.   
In addition, throughout this report I created 3 groups in 3 columns:
* Age_Group;
* Loan_Purpose_Group;
* Income_Group.
    
I renamed the existing column and analysed the data set. 

Analysis and Conclusion:
* I checked the debt rate among the clients' age group: I found that most of the clients are between the ages 40-59. **However, the age group with the highest default rate is the second highest age group in the data set, aged 20-39, at 10%.** 

* I checked the impact a client's number of children might have on his or her debt rate: I found that most of the clients in the data set have no children, and their debt rate is 7% - the second lowest rate. **However, clients with the most number of children have never reported being in debt while clients with 1-4 children have a debt rate of 8%-9%. I concluded that there is no apparnt correlation between number of children and debt rate.**

* I checked the correlation between the clients' age and debt rate: I found that clients with average income, between 20,000 and 35,000, consists the lion's share of the data set and also have the highest debt rate at 8%. The second largest income group, "Below Average Income" (with reported income between 5,000-20,000), also have the second highest debt rate - also at 8%. Clients with low income (less than 5,000) is the smallest group in the data set and have the third highest debt rate. we can note that most of the clients in the data set are either married or in another kind of partnership. **I concluded that clients' total income has an impact on their ability to repay loans on time - the higher the income, the lower debt rate.**

* I checked the correlation between a client's personal status and debt rate: I found that most of the clients in the data set are either married or in another kind of partnership. Other clients are either widowed, divorced or unmarried. Clients who are widowed reported the lowest debt rate, at 6%. However, divorced and married clients have approximately similar debt rate at 7% - as do clients who are either in a civil partnership or unmarried (9% each). **Thus, I concluded that clients' personal status has no baring on their ability to repay loans on time.** 

* I check what impact the clients' reason for taking the loan has on their ability to repay the loan, or their debt rate: I found that the lion's share of the clients took a loan for either residential or commercial real estate purposes. However, this group also recorded the lowest debt rate at 7%. The debt rate is higher for clients whose reason for taking a loan was connected either to wedding expenses, education or car financing (either purchase or repair), who also consist about half of the clients together - at 8% and 9% respectively. **Thus, I concluded that there is little correlation between a client's reason for taking a loan and his or her ability to repay the loan on time.**

**This report's final conclusion is that a client's personal status and his or her number of children have little to no impact on whether they will default on a loan, while their age (and possible their income) would.**